# Debugging

R. Burke Squires, richard.squires@nih.gov

---

Averaging over the lifetime of the project, a **programmer spends about 10-20% of his time writing code**, and most programmers write about **10-12 lines of code per day** that goes into the final product, regardless of their skill level. Good programmers spend much of the **other 90% thinking, researching, and experimenting** to find the best design. Bad programmers spend much of that 90% debugging code by randomly making changes and seeing if they work. [Mythical Man Month](http://automagical.rationalmind.net/2010/08/17/some-lesser-known-truths-about-programming/)

---

_The most effective debugging tool is still careful thought, coupled with judiciously placed print statements._
Brian Kernighan, "Unix for Beginners" (1979)

_Everyone knows that debugging is twice as hard as writing a program in the first place. So if you're as clever as you can be when you write it, how will you ever debug it?_
Brian Kernighan, "The Elements of Programming Style", 2nd edition, chapter 2

---

# Debugging...is a learned skill!

---

### After running your program and determining that it is not working...
### Can you answer these questions as best as you can?

1. What makes you say your code isn't working?
2. What did you expect your code to do and why?
3. What did your code do instead and how do you know?

---

# Debugging Methods:

1. Print statements
2. Trace
3. Logging
4. Python Debugger (PDB)
5. IDE Debugging (PyCharm, etc)

---

## 1. Print Statements

A primitive way of debugging programs is to embed print statements through out the code to track execution flow and state. However, this approach can quickly become unmaintainable for a number of reasons:
- Normal program output is mixed with debugging output. This makes it difficult to distinguish between the two.
- There is no easy way to disable debugging output, or redirect it to a file.
- When done with debugging, it may be difficult to track down and remove all print statements that are scattered over the code.

## 2. Using the Trace Module

[Using Python trace module](https://wiki.qt.io/Python_Debugging_Tips)

If your program is crashing and no information is being shown about what happened (as usually is the case with a segfault), then the Python module trace can be useful to point which line of the program is causing the crash or at least the line where the program stopped. Just add "-m trace —trace" parameters to the Python executable and every line being executed will be printed on the screen. Supposing your program is called 'script.py', one way to use trace is shown below:

In [ ]:
!python -m trace --trace calc_circle.py

Keep in mind that this will make your program run slower, as the standard output will show everything being executed at the time. For more information about the trace module, you can consult its official documentation here.

## 3. Logging

Python provides an alternative to debug print statements that doesn't suffer from the shortcomings above. This alternative comes in the form of a module called logging, and it is very powerful and easy to use.

Let's start with a simple example. The following snippet imports the logging module and sets the logging level to debug:

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)

The call to logging.basicConfig() should be done once when your program starts. Now, whenever you want to print a debug message, call logging.debug():

In [ ]:
logging.debug('This is a debug message.')

This will send the following string to stderr:

    DEBUG:root:This is a debugging message.

DEBUG indicates that this is a debug message. root indicates that this is the root logger, as it is possible to have multiple loggers (don't worry about this for now).

Now we have a better logging system that can be globally switched on and off. To turn off debug messages, simply omit the level argument when calling logging.basicConfig():

In [ ]:
logging.basicConfig()

Logging to a file and adding date/time
To take full advantage of the logging module, let's have a look at some of the options that can be provided to logging.basicConfig():

    Argument	Description
    filename	Send log messages to a file.
    filemode	The mode to open the file in (defaults to 'a').
    format	  The format of log messages.
    dateformat  date/time format in log messages.
    level	   Level of messages to be printed (more on this later).

For example, to configure the logging module to send debug messages to a file called debug.log, use:

    logging.basicConfig(level=logging.DEBUG, filename='debug.log')

Log messages will be appended to debug.log if the file already exists. This means that your log messages will be kept even if you run your program multiple times.

To add date/time to your log messages, use:

In [ ]:
logging.basicConfig(level=logging.DEBUG, filename='debug.log',
                    format='%(asctime)s %(levelname)s: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

This will result in log messages like the following:

    2009-08-30 23:30:49 DEBUG: This is a debug message.

### Logging levels

The logging supports multiple levels of log messages in addition to DEBUG. Here is the full list:


Level           |Function          
----------------|------------------
logging.CRITICAL|logging.critical()
logging.ERROR   |logging.error()   
logging.WARNING |logging.warning() 
logging.INFO    |logging.info()    
logging.DEBUG   |logging.debug()
    

Setting the logging level to a value enables log messages for this level and all levels above it. So if you set the level to logging.WARNING, you will get WARNING, ERROR and CRITICAL messages. This allows you to have different levels of log verbosity.

### Convenient template for logging

Before I conclude this section, I will provide a simple template for enabling logging functionality in your programs. This template uses command-line flags to change the level logging, which is more convenient that modifying source code.

In [ ]:
import logging
import optparse

LOGGING_LEVELS = {'critical': logging.CRITICAL,
                  'error': logging.ERROR,
                  'warning': logging.WARNING,
                  'info': logging.INFO,
                  'debug': logging.DEBUG}

def main():
    parser = optparse.OptionParser()
    parser.add_option('-l', '--logging-level', help='Logging level')
    parser.add_option('-f', '--logging-file', help='Logging file name')
    (options, args) = parser.parse_args()
    logging_level = LOGGING_LEVELS.get(options.logging_level, logging.NOTSET)
    logging.basicConfig(level=logging_level, filename=options.logging_file,
                          format='%(asctime)s %(levelname)s: %(message)s',
                          datefmt='%Y-%m-%d %H:%M:%S')

    # Your program goes here.
    # You can access command-line arguments using the args variable.

if __name__ == '__main__':
  main()

By default, the logging module prints critical, error and warning messages. To change this so that all levels are printed, use:

    $python ./your-program.py --logging=debug

To send log messages to a file called debug.log, use:

    $python ./your-program.py --logging-level=debug --logging-file=debug.log

---

## 4. Using the Python Debugger (PDB, iPDB)

When you need to examine the execution flow of your program, an interactive console is not going to be enough. For situations like this, you can use the Python debugger. It provides a full-fledged debugging environment that supports breakpoints, stepping through source code, stack inspection and much more. This debugger comes with the standard python distribution as a module named pdb.

- [PDB](https://pythonconquerstheuniverse.wordpress.com/2009/09/10/debugging-in-python/)
- [PDB cheatsheet (PDF)](http://web.stanford.edu/class/physics91si/2013/handouts/Pdb_Commands.pdf)
- [iPDB](https://www.safaribooksonline.com/blog/2014/11/18/intro-python-debugger/)

In [ ]:
# %load calc_circle_pdb.py
'''Illustrate a global constant being used inside functions.'''

import pdb

pdb.set_trace()

PI = 3.14159265358979   # global constant -- only place the value of PI is set

def circle_area(radius):
    return PI*radius*radius    # use value of global constant PI

def circle_circumference(radius):
    return 2*PI*radius         # use value of global constant PI

def main():
    print('circle area with radius 5:', circle_area(5))
    print('circumference with radius 5:', circle_circumference(5))

main()

In [ ]:
!python calc_circle_pdb.py

Note: Interupt the kernel to continue above.

---

## 5. PyCharm / spyder / IDE

Debugging
- Built-in debugger
- Set breakpoints
- Observe variables during execution

Demo

[Debugging in PyCharm](https://www.jetbrains.com/pycharm/help/debugging.html)

---

Source(s)
- [Ayman Hourieh](http://web.archive.org/web/20120819135307/http://aymanh.com/python-debugging-techniques)

Resources:
- [Debugging](https://inst.eecs.berkeley.edu/~cs61a/fa12/pdfs/debugging.pdf)
- [Teaching Novice Programmers How to Debug Their Code](http://blog.codeunion.io/2014/09/03/teaching-novices-how-to-debug-code/)
- [Computer Programming Principles/Maintaining/Debugging](https://en.wikibooks.org/wiki/Computer_Programming_Principles/Maintaining/Debugging)